In [ ]:
# JIRA to Pinecone Vector Database with Vertex AI
# This notebook connects to JIRA, extracts ticket information, and creates embeddings using Google's Vertex AI
# The embeddings are stored in Pinecone for semantic search

import os
import json
import time
from typing import List, Dict, Any, Optional
from datetime import datetime, timezone
from dataclasses import dataclass, field
from dotenv import load_dotenv

# JIRA and API imports
import requests
from requests.auth import HTTPBasicAuth

# Google Cloud and Vertex AI imports
import vertexai
from vertexai.language_models import TextEmbeddingModel

# Pinecone imports
from pinecone import Pinecone, ServerlessSpec

# Load environment variables
load_dotenv()

print("✅ Imports completed successfully!")
print("📝 Next: Configure your JIRA and Pinecone credentials in the environment variables.")

In [ ]:
@dataclass
class JiraIndexConfig:
    """Configuration for JIRA ticket indexing with customizable fields"""
    
    # JIRA Connection Settings
    jira_url: str = ""
    jira_username: str = ""
    jira_api_token: str = ""
    
    # Pinecone Settings
    pinecone_api_key: str = ""
    pinecone_index_name: str = "jira-tickets"
    pinecone_environment: str = "gcp-starter"  # or your preferred environment
    
    # Google Cloud Settings
    google_cloud_project: str = ""
    
    # Configurable JIRA Fields to Index
    fields_to_index: List[str] = field(default_factory=lambda: [
        "title",           # Issue summary/title
        "description",     # Issue description
        "summary",         # Alternative to title
        "created_date",    # When the ticket was created
        "ticket_number",   # JIRA key (e.g., PROJ-123)
        "ticket_id"        # JIRA issue ID
    ])
    
    # Additional optional fields
    optional_fields: List[str] = field(default_factory=lambda: [
        "status",
        "priority", 
        "assignee",
        "reporter",
        "issue_type",
        "components",
        "labels",
        "resolution",
        "updated_date"
    ])
    
    # JQL Query for filtering tickets
    jql_query: str = "project IN (YOUR_PROJECT) ORDER BY created DESC"
    
    # Embedding settings
    embedding_model_name: str = "text-embedding-005"
    embedding_dimension: int = 768
    
    # Batch processing settings
    batch_size: int = 50
    max_tickets: Optional[int] = None  # None for all tickets
    
    def __post_init__(self):
        """Load configuration from environment variables if not provided"""
        if not self.jira_url:
            self.jira_url = os.getenv("JIRA_URL", "")
        if not self.jira_username:
            self.jira_username = os.getenv("JIRA_USERNAME", "")
        if not self.jira_api_token:
            self.jira_api_token = os.getenv("JIRA_API_TOKEN", "")
        if not self.pinecone_api_key:
            self.pinecone_api_key = os.getenv("PINECONE_API_KEY", "")
        if not self.google_cloud_project:
            self.google_cloud_project = os.getenv("GOOGLE_CLOUD_PROJECT", "")
    
    def validate(self) -> List[str]:
        """Validate configuration and return list of missing required fields"""
        missing = []
        if not self.jira_url:
            missing.append("JIRA_URL")
        if not self.jira_username:
            missing.append("JIRA_USERNAME") 
        if not self.jira_api_token:
            missing.append("JIRA_API_TOKEN")
        if not self.pinecone_api_key:
            missing.append("PINECONE_API_KEY")
        if not self.google_cloud_project:
            missing.append("GOOGLE_CLOUD_PROJECT")
        return missing

# Create default configuration
config = JiraIndexConfig()
missing_fields = config.validate()

if missing_fields:
    print("❌ Missing required environment variables:")
    for field in missing_fields:
        print(f"   - {field}")
    print("\n💡 Please set these in your .env file or directly in the config object.")
else:
    print("✅ Configuration validated successfully!")
    print(f"📊 Will index these fields: {', '.join(config.fields_to_index)}")
    print(f"🗂️  Optional fields: {', '.join(config.optional_fields)}")

In [ ]:
class JiraClient:
    """Client for connecting to JIRA and fetching ticket data"""
    
    def __init__(self, config: JiraIndexConfig):
        self.config = config
        self.base_url = config.jira_url.rstrip('/')
        self.auth = HTTPBasicAuth(config.jira_username, config.jira_api_token)
        self.session = requests.Session()
        self.session.auth = self.auth
        
    def test_connection(self) -> bool:
        """Test JIRA connection"""
        try:
            response = self.session.get(f"{self.base_url}/rest/api/2/myself")
            if response.status_code == 200:
                user_info = response.json()
                print(f"✅ Connected to JIRA as: {user_info.get('displayName', 'Unknown')}")
                return True
            else:
                print(f"❌ JIRA connection failed: {response.status_code} - {response.text}")
                return False
        except Exception as e:
            print(f"❌ JIRA connection error: {e}")
            return False
    
    def get_ticket_count(self, jql: str) -> int:
        """Get total count of tickets matching JQL query"""
        try:
            response = self.session.get(
                f"{self.base_url}/rest/api/2/search",
                params={"jql": jql, "maxResults": 0}
            )
            if response.status_code == 200:
                return response.json().get("total", 0)
            else:
                print(f"❌ Failed to get ticket count: {response.status_code}")
                return 0
        except Exception as e:
            print(f"❌ Error getting ticket count: {e}")
            return 0
    
    def fetch_tickets_batch(self, jql: str, start_at: int = 0, max_results: int = 50) -> List[Dict[str, Any]]:
        """Fetch a batch of tickets from JIRA"""
        try:
            # Define fields to fetch (always include key fields plus configured ones)
            fields = ["key", "id", "summary", "description", "created", "updated"]
            
            # Add optional fields that are commonly available
            optional_jira_fields = [
                "status", "priority", "assignee", "reporter", 
                "issuetype", "components", "labels", "resolution"
            ]
            fields.extend(optional_jira_fields)
            
            params = {
                "jql": jql,
                "startAt": start_at,
                "maxResults": max_results,
                "fields": ",".join(fields)
            }
            
            response = self.session.get(f"{self.base_url}/rest/api/2/search", params=params)
            
            if response.status_code == 200:
                data = response.json()
                return data.get("issues", [])
            else:
                print(f"❌ Failed to fetch tickets: {response.status_code} - {response.text}")
                return []
                
        except Exception as e:
            print(f"❌ Error fetching tickets: {e}")
            return []
    
    def extract_ticket_data(self, jira_issue: Dict[str, Any]) -> Dict[str, Any]:
        """Extract and normalize ticket data based on configuration"""
        fields = jira_issue.get("fields", {})
        ticket_data = {}
        
        # Map JIRA fields to our configured field names
        field_mapping = {
            "ticket_number": jira_issue.get("key", ""),
            "ticket_id": jira_issue.get("id", ""),
            "title": fields.get("summary", ""),
            "summary": fields.get("summary", ""),  # Alternative name for title
            "description": fields.get("description", "") or "",
            "created_date": fields.get("created", ""),
            "updated_date": fields.get("updated", ""),
            "status": fields.get("status", {}).get("name", "") if fields.get("status") else "",
            "priority": fields.get("priority", {}).get("name", "") if fields.get("priority") else "",
            "assignee": fields.get("assignee", {}).get("displayName", "") if fields.get("assignee") else "Unassigned",
            "reporter": fields.get("reporter", {}).get("displayName", "") if fields.get("reporter") else "",
            "issue_type": fields.get("issuetype", {}).get("name", "") if fields.get("issuetype") else "",
            "components": ", ".join([c.get("name", "") for c in fields.get("components", [])]),
            "labels": ", ".join(fields.get("labels", [])),
            "resolution": fields.get("resolution", {}).get("name", "") if fields.get("resolution") else ""
        }
        
        # Include only configured fields in the final data
        all_configured_fields = self.config.fields_to_index + self.config.optional_fields
        for field_name in all_configured_fields:
            if field_name in field_mapping:
                ticket_data[field_name] = field_mapping[field_name]
        
        return ticket_data
    
    def create_text_content(self, ticket_data: Dict[str, Any]) -> str:
        """Create searchable text content from ticket data"""
        # Prioritize the configured fields for text content
        content_parts = []
        
        # Always include title/summary first if available
        if ticket_data.get("title") or ticket_data.get("summary"):
            title = ticket_data.get("title") or ticket_data.get("summary")
            content_parts.append(f"Title: {title}")
        
        # Add description if configured and available
        if "description" in self.config.fields_to_index and ticket_data.get("description"):
            content_parts.append(f"Description: {ticket_data['description']}")
        
        # Add other configured fields
        for field in self.config.fields_to_index:
            if field in ["title", "summary", "description"]:
                continue  # Already handled above
            if ticket_data.get(field):
                field_label = field.replace("_", " ").title()
                content_parts.append(f"{field_label}: {ticket_data[field]}")
        
        return " | ".join(content_parts)

# Initialize JIRA client
if not missing_fields:
    jira_client = JiraClient(config)
    if jira_client.test_connection():
        print("🎉 JIRA client initialized successfully!")
    else:
        print("❌ Failed to initialize JIRA client")
else:
    print("⏸️ Skipping JIRA client initialization due to missing configuration")

In [ ]:
class VertexAIEmbeddingClient:
    """Client for generating embeddings using Google's Vertex AI"""
    
    def __init__(self, config: JiraIndexConfig):
        self.config = config
        self.embedding_model = None
        self.initialize_vertex_ai()
    
    def initialize_vertex_ai(self):
        """Initialize Vertex AI and embedding model"""
        try:
            # Initialize Vertex AI with project
            vertexai.init(project=self.config.google_cloud_project)
            
            # Load the embedding model
            self.embedding_model = TextEmbeddingModel.from_pretrained(self.config.embedding_model_name)
            
            print(f"✅ Vertex AI initialized with model: {self.config.embedding_model_name}")
            print(f"🏗️ Project: {self.config.google_cloud_project}")
            
        except Exception as e:
            print(f"❌ Failed to initialize Vertex AI: {e}")
            print("💡 Make sure you have set up Google Cloud authentication and have the correct project ID")
    
    def test_embedding(self) -> bool:
        """Test embedding generation with a sample text"""
        if not self.embedding_model:
            print("❌ Embedding model not initialized")
            return False
        
        try:
            test_text = "This is a test ticket about login issues"
            embeddings = self.embedding_model.get_embeddings([test_text])
            
            if embeddings and len(embeddings) > 0:
                embedding_vector = embeddings[0].values
                print(f"✅ Embedding test successful!")
                print(f"📊 Embedding dimension: {len(embedding_vector)}")
                print(f"🔢 Sample values: {embedding_vector[:5]}...")
                return True
            else:
                print("❌ Embedding test failed: No embeddings returned")
                return False
                
        except Exception as e:
            print(f"❌ Embedding test failed: {e}")
            return False
    
    def generate_embeddings_batch(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for a batch of texts"""
        if not self.embedding_model:
            raise Exception("Embedding model not initialized")
        
        try:
            # Vertex AI embedding model can handle multiple texts at once
            embeddings = self.embedding_model.get_embeddings(texts)
            return [embedding.values for embedding in embeddings]
            
        except Exception as e:
            print(f"❌ Error generating embeddings: {e}")
            return []
    
    def generate_single_embedding(self, text: str) -> List[float]:
        """Generate embedding for a single text"""
        embeddings = self.generate_embeddings_batch([text])
        return embeddings[0] if embeddings else []

# Initialize Vertex AI client
if not missing_fields:
    vertex_client = VertexAIEmbeddingClient(config)
    if vertex_client.test_embedding():
        print("🎉 Vertex AI embedding client ready!")
    else:
        print("❌ Vertex AI embedding client initialization failed")
else:
    print("⏸️ Skipping Vertex AI client initialization due to missing configuration")

In [ ]:
class PineconeClient:
    """Client for Pinecone vector database operations"""
    
    def __init__(self, config: JiraIndexConfig):
        self.config = config
        self.pc = None
        self.index = None
        self.initialize_pinecone()
    
    def initialize_pinecone(self):
        """Initialize Pinecone client and connect to index"""
        try:
            # Initialize Pinecone
            self.pc = Pinecone(api_key=self.config.pinecone_api_key)
            print("✅ Pinecone client initialized")
            
        except Exception as e:
            print(f"❌ Failed to initialize Pinecone: {e}")
    
    def create_index_if_not_exists(self) -> bool:
        """Create Pinecone index if it doesn't exist"""
        if not self.pc:
            print("❌ Pinecone client not initialized")
            return False
        
        try:
            # Check if index already exists
            existing_indexes = [idx.name for idx in self.pc.list_indexes()]
            
            if self.config.pinecone_index_name in existing_indexes:
                print(f"✅ Index '{self.config.pinecone_index_name}' already exists")
                self.index = self.pc.Index(self.config.pinecone_index_name)
                return True
            
            # Create new index
            print(f"🏗️ Creating new index: {self.config.pinecone_index_name}")
            self.pc.create_index(
                name=self.config.pinecone_index_name,
                dimension=self.config.embedding_dimension,
                metric="cosine",
                spec=ServerlessSpec(
                    cloud="gcp",  # or "aws" 
                    region="us-central1"  # adjust based on your preference
                )
            )
            
            # Wait for index to be ready
            print("⏳ Waiting for index to be ready...")
            time.sleep(30)  # Give index time to initialize
            
            self.index = self.pc.Index(self.config.pinecone_index_name)
            print(f"✅ Index '{self.config.pinecone_index_name}' created successfully!")
            
            return True
            
        except Exception as e:
            print(f"❌ Error creating index: {e}")
            return False
    
    def get_index_stats(self) -> Dict[str, Any]:
        """Get statistics about the index"""
        if not self.index:
            return {}
        
        try:
            stats = self.index.describe_index_stats()
            return stats
        except Exception as e:
            print(f"❌ Error getting index stats: {e}")
            return {}
    
    def upsert_vectors_batch(self, vectors_data: List[Dict[str, Any]]) -> bool:
        """Upsert a batch of vectors to Pinecone"""
        if not self.index:
            print("❌ Index not initialized")
            return False
        
        try:
            # Prepare vectors for upsert
            vectors_to_upsert = []
            for data in vectors_data:
                vector_id = data.get("id")
                embedding = data.get("embedding")
                metadata = data.get("metadata", {})
                
                if vector_id and embedding:
                    vectors_to_upsert.append({
                        "id": vector_id,
                        "values": embedding,
                        "metadata": metadata
                    })
            
            if vectors_to_upsert:
                self.index.upsert(vectors=vectors_to_upsert)
                print(f"✅ Upserted {len(vectors_to_upsert)} vectors to Pinecone")
                return True
            else:
                print("⚠️ No valid vectors to upsert")
                return False
                
        except Exception as e:
            print(f"❌ Error upserting vectors: {e}")
            return False
    
    def query_similar_vectors(self, query_embedding: List[float], top_k: int = 5) -> List[Dict[str, Any]]:
        """Query for similar vectors"""
        if not self.index:
            print("❌ Index not initialized")
            return []
        
        try:
            results = self.index.query(
                vector=query_embedding,
                top_k=top_k,
                include_metadata=True,
                include_values=False
            )
            
            return [{
                "id": match.id,
                "score": match.score,
                "metadata": match.metadata
            } for match in results.matches]
            
        except Exception as e:
            print(f"❌ Error querying vectors: {e}")
            return []
    
    def delete_all_vectors(self) -> bool:
        """Delete all vectors from the index (use with caution!)"""
        if not self.index:
            print("❌ Index not initialized")
            return False
        
        try:
            self.index.delete(delete_all=True)
            print("✅ All vectors deleted from index")
            return True
        except Exception as e:
            print(f"❌ Error deleting vectors: {e}")
            return False

# Initialize Pinecone client
if not missing_fields:
    pinecone_client = PineconeClient(config)
    if pinecone_client.create_index_if_not_exists():
        stats = pinecone_client.get_index_stats()
        print(f"📊 Index stats: {stats}")
        print("🎉 Pinecone client ready!")
    else:
        print("❌ Pinecone client initialization failed")
else:
    print("⏸️ Skipping Pinecone client initialization due to missing configuration")

In [ ]:
class JiraPineconeIndexer:
    """Main orchestrator for indexing JIRA tickets to Pinecone"""
    
    def __init__(self, config: JiraIndexConfig):
        self.config = config
        self.jira_client = JiraClient(config)
        self.vertex_client = VertexAIEmbeddingClient(config)
        self.pinecone_client = PineconeClient(config)
        
    def validate_setup(self) -> bool:
        """Validate that all clients are properly initialized"""
        checks = [
            ("JIRA", self.jira_client.test_connection()),
            ("Vertex AI", self.vertex_client.embedding_model is not None),
            ("Pinecone", self.pinecone_client.index is not None)
        ]
        
        all_good = True
        for service, status in checks:
            if status:
                print(f"✅ {service}: Ready")
            else:
                print(f"❌ {service}: Not ready")
                all_good = False
        
        return all_good
    
    def preview_tickets(self, max_preview: int = 5) -> List[Dict[str, Any]]:
        """Preview a few tickets to see what data will be indexed"""
        print(f"🔍 Previewing {max_preview} tickets with current configuration...")
        print(f"📋 Fields to index: {', '.join(self.config.fields_to_index)}")
        print("-" * 60)
        
        tickets = self.jira_client.fetch_tickets_batch(
            self.config.jql_query, 
            start_at=0, 
            max_results=max_preview
        )
        
        previews = []
        for i, ticket in enumerate(tickets, 1):
            ticket_data = self.jira_client.extract_ticket_data(ticket)
            text_content = self.jira_client.create_text_content(ticket_data)
            
            print(f"\n🎫 Ticket {i}: {ticket_data.get('ticket_number', 'Unknown')}")
            print(f"📝 Text content preview:")
            print(f"   {text_content[:200]}{'...' if len(text_content) > 200 else ''}")
            print(f"🏷️ Metadata fields: {list(ticket_data.keys())}")
            
            previews.append({
                'ticket_data': ticket_data,
                'text_content': text_content
            })
        
        return previews
    
    def index_all_tickets(self, dry_run: bool = False) -> Dict[str, Any]:
        """Index all JIRA tickets to Pinecone"""
        if not self.validate_setup():
            return {"success": False, "error": "Setup validation failed"}
        
        print("🚀 Starting JIRA ticket indexing process...")
        
        # Get total ticket count
        total_tickets = self.jira_client.get_ticket_count(self.config.jql_query)
        print(f"📊 Total tickets to process: {total_tickets}")
        
        if self.config.max_tickets:
            total_tickets = min(total_tickets, self.config.max_tickets)
            print(f"⚡ Limited to: {total_tickets} tickets")
        
        if dry_run:
            print("🧪 DRY RUN MODE - No data will be written to Pinecone")
        
        # Process tickets in batches
        start_time = time.time()
        processed_count = 0
        error_count = 0
        batch_count = 0
        
        for start_at in range(0, total_tickets, self.config.batch_size):
            batch_count += 1
            end_at = min(start_at + self.config.batch_size, total_tickets)
            
            print(f"\n📦 Processing batch {batch_count}: tickets {start_at + 1}-{end_at}")
            
            # Fetch ticket batch
            tickets = self.jira_client.fetch_tickets_batch(
                self.config.jql_query,
                start_at=start_at,
                max_results=self.config.batch_size
            )
            
            if not tickets:
                print("⚠️ No tickets returned, stopping")
                break
            
            # Process batch
            batch_vectors = []
            for ticket in tickets:
                try:
                    # Extract ticket data
                    ticket_data = self.jira_client.extract_ticket_data(ticket)
                    text_content = self.jira_client.create_text_content(ticket_data)
                    
                    if not text_content.strip():
                        print(f"⚠️ Skipping {ticket_data.get('ticket_number', 'Unknown')}: No content")
                        continue
                    
                    # Generate embedding
                    embedding = self.vertex_client.generate_single_embedding(text_content)
                    
                    if not embedding:
                        print(f"⚠️ Failed to generate embedding for {ticket_data.get('ticket_number', 'Unknown')}")
                        error_count += 1
                        continue
                    
                    # Prepare vector data
                    vector_data = {
                        "id": ticket_data.get("ticket_number", f"ticket-{ticket_data.get('ticket_id', 'unknown')}"),
                        "embedding": embedding,
                        "metadata": {
                            **ticket_data,
                            "text_content": text_content[:1000],  # Limit metadata size
                            "indexed_at": datetime.now(timezone.utc).isoformat()
                        }
                    }
                    
                    batch_vectors.append(vector_data)
                    processed_count += 1
                    
                except Exception as e:
                    print(f"❌ Error processing ticket: {e}")
                    error_count += 1
            
            # Upsert batch to Pinecone
            if batch_vectors and not dry_run:
                success = self.pinecone_client.upsert_vectors_batch(batch_vectors)
                if not success:
                    print(f"❌ Failed to upsert batch {batch_count}")
            elif dry_run:
                print(f"🧪 Would upsert {len(batch_vectors)} vectors (dry run)")
            
            # Progress update
            elapsed = time.time() - start_time
            rate = processed_count / elapsed if elapsed > 0 else 0
            print(f"📈 Progress: {processed_count}/{total_tickets} processed ({rate:.1f} tickets/sec)")
        
        # Final summary
        total_time = time.time() - start_time
        summary = {
            "success": True,
            "total_processed": processed_count,
            "total_errors": error_count,
            "total_time_seconds": total_time,
            "processing_rate": processed_count / total_time if total_time > 0 else 0,
            "dry_run": dry_run
        }
        
        print(f"\n🎉 Indexing complete!")
        print(f"✅ Processed: {processed_count} tickets")
        print(f"❌ Errors: {error_count}")
        print(f"⏱️ Total time: {total_time:.1f} seconds")
        print(f"📊 Rate: {summary['processing_rate']:.1f} tickets/second")
        
        if not dry_run:
            # Get final index stats
            final_stats = self.pinecone_client.get_index_stats()
            print(f"📈 Final index stats: {final_stats}")
        
        return summary
    
    def test_search(self, query: str, top_k: int = 3) -> List[Dict[str, Any]]:
        """Test search functionality with a query"""
        if not self.validate_setup():
            return []
        
        print(f"🔍 Testing search with query: '{query}'")
        
        # Generate query embedding
        query_embedding = self.vertex_client.generate_single_embedding(query)
        if not query_embedding:
            print("❌ Failed to generate query embedding")
            return []
        
        # Search Pinecone
        results = self.pinecone_client.query_similar_vectors(query_embedding, top_k)
        
        print(f"📋 Found {len(results)} similar tickets:")
        for i, result in enumerate(results, 1):
            metadata = result.get("metadata", {})
            print(f"\n{i}. {metadata.get('ticket_number', 'Unknown')} (Score: {result['score']:.3f})")
            print(f"   📝 {metadata.get('title', metadata.get('summary', 'No title'))}")
            print(f"   🎯 {metadata.get('text_content', '')[:100]}...")
        
        return results

# Initialize the main indexer
if not missing_fields:
    indexer = JiraPineconeIndexer(config)
    print("🎉 JIRA Pinecone Indexer initialized successfully!")
    print("💡 You can now run indexer.preview_tickets() to see what will be indexed")
else:
    print("⏸️ Skipping indexer initialization due to missing configuration")

# 🚀 JIRA to Pinecone Vector Database Setup Complete!

## ✅ What We've Built

This notebook provides a complete solution for:

1. **🔗 JIRA Connection**: Connects to JIRA using REST API with configurable authentication
2. **🧠 Vertex AI Embeddings**: Uses Google's `text-embedding-005` model for high-quality embeddings  
3. **🗂️ Pinecone Vector DB**: Stores and indexes ticket embeddings for semantic search
4. **⚙️ Configurable Fields**: Customize which JIRA fields to index (title, description, summary, created_date, ticket_number, etc.)

## 🛠️ Setup Instructions

### 1. Environment Variables Required

Create a `.env` file in your project root with:

```bash
# JIRA Configuration
JIRA_URL=https://your-domain.atlassian.net
JIRA_USERNAME=your-email@company.com  
JIRA_API_TOKEN=your-jira-api-token

# Pinecone Configuration
PINECONE_API_KEY=your-pinecone-api-key
PINECONE_INDEX_NAME=jira-tickets

# Google Cloud Configuration  
GOOGLE_CLOUD_PROJECT=your-gcp-project-id
```

### 2. Field Configuration

Customize which JIRA fields to index by modifying the `config.fields_to_index` list:

```python
# Default configured fields
config.fields_to_index = [
    "title",           # Issue summary/title
    "description",     # Issue description  
    "summary",         # Alternative to title
    "created_date",    # When ticket was created
    "ticket_number",   # JIRA key (e.g., PROJ-123)
    "ticket_id"        # JIRA issue ID
]

# Optional additional fields
config.optional_fields = [
    "status", "priority", "assignee", "reporter", 
    "issue_type", "components", "labels", "resolution", "updated_date"
]
```

### 3. JQL Query Customization

Modify the JQL query to filter which tickets to index:

```python
config.jql_query = "project IN (MYPROJ, OTHERPROJ) AND created >= -30d ORDER BY created DESC"
```

## 🎯 Usage Examples

The next cells will show you how to use the indexer!

In [ ]:
# 📋 Step 1: Customize Configuration (Optional)
# Modify these settings based on your needs

if 'config' in locals():
    # Example: Customize fields to index
    config.fields_to_index = [
        "title",
        "description", 
        "ticket_number",
        "created_date",
        "summary"
    ]
    
    # Example: Customize JQL query
    config.jql_query = "project = YOUR_PROJECT ORDER BY created DESC"
    
    # Example: Limit number of tickets for testing
    config.max_tickets = 100  # Remove or set to None for all tickets
    
    print("✅ Configuration updated!")
    print(f"🎯 Fields to index: {config.fields_to_index}")
    print(f"🔍 JQL Query: {config.jql_query}")
    if config.max_tickets:
        print(f"📊 Max tickets: {config.max_tickets}")
else:
    print("⚠️ Configuration not available - check previous cells for errors")

In [ ]:
# 🔍 Step 2: Preview Tickets
# See what tickets will be indexed and how they'll be processed

if 'indexer' in locals():
    print("🔍 Previewing tickets that will be indexed...")
    preview_results = indexer.preview_tickets(max_preview=3)
    
    print(f"\n📊 Preview complete! {len(preview_results)} tickets shown.")
    print("💡 If this looks good, proceed to the next step to start indexing.")
else:
    print("⚠️ Indexer not available - check previous cells for errors")

In [ ]:
# 🧪 Step 3: Dry Run (Recommended)
# Test the full process without writing to Pinecone

if 'indexer' in locals():
    print("🧪 Running dry run - no data will be written to Pinecone")
    print("This helps validate the process before running the real indexing")
    
    dry_run_results = indexer.index_all_tickets(dry_run=True)
    
    if dry_run_results.get("success"):
        print("\n✅ Dry run completed successfully!")
        print(f"📊 Would process {dry_run_results['total_processed']} tickets")
        print(f"❌ {dry_run_results['total_errors']} errors encountered")
        print(f"⏱️ Estimated time: {dry_run_results['total_time_seconds']:.1f} seconds")
        print("🚀 Ready to run the real indexing!")
    else:
        print("❌ Dry run failed:", dry_run_results.get("error", "Unknown error"))
else:
    print("⚠️ Indexer not available - check previous cells for errors")

In [ ]:
# 🚀 Step 4: Run Real Indexing
# ⚠️ This will actually create embeddings and store them in Pinecone

if 'indexer' in locals():
    print("🚀 Starting real indexing process...")
    print("⚠️ This will consume Vertex AI API credits and store data in Pinecone")
    
    # Uncomment the line below when you're ready to run the real indexing
    # results = indexer.index_all_tickets(dry_run=False)
    
    print("🛑 Real indexing is commented out for safety.")
    print("💡 Uncomment the line above when you're ready to proceed.")
    print("📋 Make sure you've reviewed the dry run results first!")
    
    # If you want to run it, uncomment these lines:
    # if results.get("success"):
    #     print(f"✅ Indexing completed! Processed {results['total_processed']} tickets")
    # else:
    #     print(f"❌ Indexing failed: {results.get('error', 'Unknown error')}")
else:
    print("⚠️ Indexer not available - check previous cells for errors")

In [ ]:
# 🔍 Step 5: Test Search Functionality
# Test semantic search on your indexed tickets

if 'indexer' in locals():
    # Example search queries - modify these based on your tickets
    test_queries = [
        "login authentication issues",
        "database performance slow",
        "UI button overlapping mobile",
        "API endpoint returning error",
        "security vulnerability file upload"
    ]
    
    print("🔍 Testing search functionality with sample queries...")
    print("💡 These queries use semantic similarity to find relevant tickets")
    print("-" * 60)
    
    for query in test_queries:
        print(f"\n🎯 Testing query: '{query}'")
        results = indexer.test_search(query, top_k=2)
        
        if not results:
            print("   📭 No results found")
        print("   " + "-" * 50)
    
    print("\n✅ Search testing complete!")
    print("💡 You can now use indexer.test_search('your query') for custom searches")
    
else:
    print("⚠️ Indexer not available - check previous cells for errors")

# 🎉 JIRA to Pinecone Vector Database Complete!

## ✅ What You've Accomplished

1. **🔗 Connected to JIRA** - Authenticated and tested connection to your JIRA instance
2. **🧠 Configured Vertex AI** - Set up Google's text-embedding-005 model for embeddings
3. **🗂️ Set up Pinecone** - Created/connected to vector database for storage
4. **⚙️ Customized Fields** - Configured which JIRA ticket fields to index
5. **🔍 Tested Search** - Validated semantic search functionality

## 🎯 Configurable Fields Used

The indexer is configured to use these JIRA ticket fields:

- **title/summary** - The main ticket title
- **description** - Detailed ticket description  
- **ticket_number** - JIRA key (e.g., PROJ-123)
- **ticket_id** - Internal JIRA ID
- **created_date** - When the ticket was created
- Plus optional fields like status, priority, assignee, etc.

## 🚀 Next Steps

### For CrewAI Integration:
Your existing `JiraTicketSearchTool` in `src/cria_crew/tools/custom_tool.py` is already set up to use this Pinecone index!

### For Production Use:
1. **Schedule Updates** - Set up periodic re-indexing for new tickets
2. **Monitor Usage** - Track Vertex AI API usage and Pinecone storage
3. **Optimize Queries** - Fine-tune JQL queries for your specific needs
4. **Scale Up** - Remove `max_tickets` limit for full indexing

### Custom Search:
```python
# Use this for custom searches
results = indexer.test_search("your search query", top_k=5)
```

### Environment Variables Required:
```bash
JIRA_URL=https://your-domain.atlassian.net
JIRA_USERNAME=your-email@company.com
JIRA_API_TOKEN=your-jira-api-token  
PINECONE_API_KEY=your-pinecone-api-key
GOOGLE_CLOUD_PROJECT=your-gcp-project-id
```

🎊 **Your JIRA tickets are now searchable using semantic similarity!**